In [1]:
import nltk
import numpy as np
import pandas as pd
from nltk import ngrams
from nltk.probability import FreqDist
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Step 1: Load the data
data = pd.read_csv('/content/drive/MyDrive/NLP/CA1_Codes/data/google_play_store_apps_reviews.csv')

# Step 2: Split the data
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42)

In [4]:
train_data

,package_name,review,polarity
331,com.viber.voip,just did the latest update on viber and yet ...,0
733,com.opera.mini.native,keeps crashing it only works well in extreme ...,0
382,com.google.android.talk,the fail boat has arrived the 6.0 version is ...,0
704,com.opera.mini.native,"superfast, just as i remember it ! opera mini...",1
813,org.mozilla.firefox,installed and immediately deleted this crap i...,1
...,...,...,...
106,com.linkedin.android,why can't i share my achievements? recently d...,0
270,com.android.chrome,beta is the best version of the chrome browse...,0
860,com.rovio.angrybirds,great little game. this is a great little gam...,1
435,com.facebook.orca,keeps crashing ever since i started using it ...,0


In [5]:
# Step 3: Build the n-gram Language Model
def get_ngrams(text, n):
    tokens = nltk.word_tokenize(text)
    return list(ngrams(tokens, n))

def train_ngram(data, n):
    positive_ngrams = []
    negative_ngrams = []

    for index, row in data.iterrows():
        grams = get_ngrams(row['review'], n)
        if row['polarity'] == 1:
            positive_ngrams.extend(grams)
        elif row['polarity'] == 0:
            negative_ngrams.extend(grams)

    positive_freq = FreqDist(positive_ngrams)
    negative_freq = FreqDist(negative_ngrams)

    return positive_freq, negative_freq

# Step 4: Train the Model
n = 2  # Change to the desired n-gram size
positive_freq, negative_freq = train_ngram(train_data, n)

In [9]:
# Step 5: test the n-gram
def test_ngram(data, positive_freq, negative_freq, n):
  pred_labels = []

  # Implement testing n-gram language model process here.
  for index, row in data.iterrows():
    grams = get_ngrams(row['review'], n)
    positive_score = sum(positive_freq[gram] for gram in grams)
    negative_score = sum(negative_freq[gram] for gram in grams)

    # Classify as positive if the positive score is greater, else classify as negative
    if positive_score > negative_score:
        pred_labels.append(1)
    else:
        pred_labels.append(0)

  return pred_labels

In [10]:
# Step 6: Evaluate the model on the test set
true_labels = test_data['polarity'].values
pred_labels = test_ngram(test_data, positive_freq, negative_freq, n)
# Implement the evaluation process here.
accuracy_score(true_labels, pred_labels)

0.7653631284916201